In [1]:
import pandas as pd
import numpy as np

In [2]:
#!pip install cvxpy

In [3]:
#import cvxpy as cp

# heli

# Read in finish probabilities

### Option 1: scrape from website

In [4]:
# import requests

# page = requests.get('https://datagolf.ca/pga-tour-predictions')
# page.status_code

# info = str(page.content) 
# #use str(page.content), not page.text.  Otherwise ast.literal_eval doesn't work
# #info[:100]

# #Can we just find the dict we care about?
# #data we want looks like:
# # var probs = JSON.parse('[{....}]')
# import re
# d = re.findall("var probs = JSON\.parse(.*?)]",info)
# #type(d)

# import ast
# e = ast.literal_eval(str(d[0])[5:])
# #type(e)

In [5]:
# #Now build out df
# df2 = pd.DataFrame(e[0],index=np.arange(1))
# df2.head()

In [6]:
# for i in range(1,len(e)):
#     p = e[i]
#     df2 = pd.concat([df2,pd.DataFrame(e[i],index=np.arange(1))])
# df2.head()

In [7]:
# #delete columns we don't care about
# df2.drop(['event_name','flag','sample_size'],axis=1,inplace=True)
# df2.head(2)

In [8]:
# df2.head(20)

### Option 2: Import 

In [9]:
euro = True

In [10]:
if euro:
    df2=pd.read_csv('Data/Euro_preds.csv')
else:
    df2= pd.read_csv('Data/pga_preds.csv')
df2.head()

,player_name,sample_size,win,top_5,top_10,top_20,make_cut
0,"Coetzee, George",150,0.131400,0.350950,0.494600,0.655750,0.886375
1,"Vincent, Scott",150,0.038925,0.152925,0.258550,0.416100,0.783300
2,"Higgo, Garrick",150,0.038000,0.148475,0.252000,0.409475,0.782775
3,"Ormsby, Wade",150,0.033725,0.141625,0.242525,0.399775,0.780950
4,"Donaldson, Jamie",150,0.028225,0.122125,0.214625,0.361150,0.757000


In [11]:
#Add field with name of First Last
if euro:
    df2['temp_Name'] = df2.player_name
    temp_name = df2.player_name.str.split(', ',expand=True,n=2)
    df2['temp_Name'] = temp_name[1] + " " + temp_name[0]
else:
    df2['temp_Name'] = df2.player_name
    temp_name = df2.player_name.str.split(', ',expand=True,n=2)
    df2['temp_Name'] = temp_name[1] + " " + temp_name[0]
df2.head()

,player_name,sample_size,win,top_5,top_10,top_20,make_cut,temp_Name
0,"Coetzee, George",150,0.131400,0.350950,0.494600,0.655750,0.886375,George Coetzee
1,"Vincent, Scott",150,0.038925,0.152925,0.258550,0.416100,0.783300,Scott Vincent
2,"Higgo, Garrick",150,0.038000,0.148475,0.252000,0.409475,0.782775,Garrick Higgo
3,"Ormsby, Wade",150,0.033725,0.141625,0.242525,0.399775,0.780950,Wade Ormsby
4,"Donaldson, Jamie",150,0.028225,0.122125,0.214625,0.361150,0.757000,Jamie Donaldson


In [12]:
#Add sal
#df2_sal = pd.read_csv('Data/DKSalaries Memorial 2019.csv')
if euro:
    url = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=29&draftGroupId=39816'
else:
    url = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=29&draftGroupId=30976'
#url='https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=29&draftGroupId=39678'
df2_sal = pd.read_csv(url,
                     encoding='latin')
#df2_sal = pd.read_csv('Data/DKSalaries_euro')
df2_sal.head(2)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,George Coetzee (15413332),George Coetzee,15413332,G,11500,Open de Portugal at Royal Óbidos,Golf,78.36
1,G,Garrick Higgo (15413333),Garrick Higgo,15413333,G,10900,Open de Portugal at Royal Óbidos,Golf,75.38


In [13]:
#To make sure names match, replace all names in df2 with actual DK names in df2_sal
from difflib import get_close_matches
df2['Name'] = ""
for i in np.arange(len(df2)):
    curr_name = df2['temp_Name'].iloc[i]
    try:
        new_name = get_close_matches(curr_name,df2_sal['Name'],cutoff=.8)[0]
        if new_name != curr_name:
            print("{} replaced with {}".format(curr_name,new_name))
    except:
        new_name = 'No Name'
        print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
    df2['Name'].iloc[i] = new_name
#df2['Name'] = df2['Name'].apply(lambda x: get_close_matches(x,df2_sal['Name']))
df2.head(5)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


NO MATCH: curr_name = Ashun Wu, new_name = No Name
Gonzalo Fdez-Castano replaced with Gonzalo Fernandez-Castano
Pep Angles Ros replaced with Pep Angles
Martin Leth Simonsen replaced with Martin Simonsen
NO MATCH: curr_name = Tyler Koivisto, new_name = No Name
Bernard Ritthammer replaced with Bernd Ritthammer
Santiago Tarrio Ben replaced with Santiago Tarrio
Lorenzo Filipo Scalise replaced with Lorenzo Scalise
Jbe' Kruger replaced with Jbe Kruger
Frederic LaCroix replaced with Frederic Lacroix
Jonathan Thomson replaced with Jon Thomson
NO MATCH: curr_name = Alexandre Daydou, new_name = No Name
NO MATCH: curr_name = Tomas Bessa, new_name = No Name
Pedro Lencart silva  replaced with Pedro Lencart Silva
Tomas Melo Gouveia replaced with Tomas Gouveia


,player_name,sample_size,win,top_5,top_10,top_20,make_cut,temp_Name,Name
0,"Coetzee, George",150,0.131400,0.350950,0.494600,0.655750,0.886375,George Coetzee,George Coetzee
1,"Vincent, Scott",150,0.038925,0.152925,0.258550,0.416100,0.783300,Scott Vincent,Scott Vincent
2,"Higgo, Garrick",150,0.038000,0.148475,0.252000,0.409475,0.782775,Garrick Higgo,Garrick Higgo
3,"Ormsby, Wade",150,0.033725,0.141625,0.242525,0.399775,0.780950,Wade Ormsby,Wade Ormsby
4,"Donaldson, Jamie",150,0.028225,0.122125,0.214625,0.361150,0.757000,Jamie Donaldson,Jamie Donaldson


In [14]:
#Find where DK Name ('Name') doesn't match the name from datagolf ('temp_Name')
df2[df2.Name == "No Name"]

,player_name,sample_size,win,top_5,top_10,top_20,make_cut,temp_Name,Name
7,"Wu, Ashun",150,0.022200,0.102325,0.186725,0.325700,0.734150,Ashun Wu,No Name
21,"Koivisto, Tyler",4,0.012250,0.065400,0.126875,0.241725,0.676025,Tyler Koivisto,No Name
109,"Daydou, Alexandre",150,0.000575,0.005825,0.017275,0.047575,0.420225,Alexandre Daydou,No Name
119,"Bessa, Tomas",60,0.000025,0.000475,0.001675,0.006975,0.223675,Tomas Bessa,No Name


In [15]:
#Add salary and  to df2
df2 = pd.merge(df2,df2_sal[['Name','Name + ID','Salary']],how='left',on='Name')
df2.dropna(inplace=True)
df2.head()

,player_name,sample_size,win,top_5,top_10,top_20,make_cut,temp_Name,Name,Name + ID,Salary
0,"Coetzee, George",150,0.131400,0.350950,0.494600,0.655750,0.886375,George Coetzee,George Coetzee,George Coetzee (15413332),11500.0
1,"Vincent, Scott",150,0.038925,0.152925,0.258550,0.416100,0.783300,Scott Vincent,Scott Vincent,Scott Vincent (15413334),10500.0
2,"Higgo, Garrick",150,0.038000,0.148475,0.252000,0.409475,0.782775,Garrick Higgo,Garrick Higgo,Garrick Higgo (15413333),10900.0
3,"Ormsby, Wade",150,0.033725,0.141625,0.242525,0.399775,0.780950,Wade Ormsby,Wade Ormsby,Wade Ormsby (15413337),9700.0
4,"Donaldson, Jamie",150,0.028225,0.122125,0.214625,0.361150,0.757000,Jamie Donaldson,Jamie Donaldson,Jamie Donaldson (15413335),10300.0


In [16]:
#Copy
df_all = df2.copy()

#Remove extra name columns
df_all  = df_all[df_all.columns.difference(['temp_Name','player_name'])]
df_all.sort_values(by='Salary',ascending=False).head(10)

,Name,Name + ID,Salary,make_cut,sample_size,top_10,top_20,top_5,win
0,George Coetzee,George Coetzee (15413332),11500.0,0.886375,150,0.494600,0.655750,0.350950,0.131400
2,Garrick Higgo,Garrick Higgo (15413333),10900.0,0.782775,150,0.252000,0.409475,0.148475,0.038000
1,Scott Vincent,Scott Vincent (15413334),10500.0,0.783300,150,0.258550,0.416100,0.152925,0.038925
4,Jamie Donaldson,Jamie Donaldson (15413335),10300.0,0.757000,150,0.214625,0.361150,0.122125,0.028225
44,Bjorn Hellgren,Bjorn Hellgren (15413336),10100.0,0.632750,150,0.089850,0.190650,0.043000,0.007350
3,Wade Ormsby,Wade Ormsby (15413337),9700.0,0.780950,150,0.242525,0.399775,0.141625,0.033725
6,Aaron Cockerill,Aaron Cockerill (15413338),9500.0,0.735650,150,0.188200,0.328100,0.102750,0.023175
25,Craig Howie,Craig Howie (15413339),9300.0,0.671675,147,0.117375,0.230925,0.058100,0.011200
20,Niklas Norgaard Moller,Niklas Norgaard Moller (15413340),9200.0,0.686675,150,0.134350,0.253800,0.068050,0.013000
10,Gonzalo Fernandez-Castano,Gonzalo Fernandez-Castano (15413341),9100.0,0.719225,150,0.164575,0.301950,0.087925,0.017825


In [17]:
df_all.to_csv('Data/Euro_preds2.csv')

# Optimize

In [15]:
#Set team parameters
lock = ['Christiaan Bezuidenhout','Thomas Detry']
#lock = ['Jon Rahm','Rory McIlroy']
lock = []

lock_full_names = []
for i, p in enumerate(lock):
    lock_full_names.append(df_all[df_all.Name==p]['Name + ID'].iloc[0])
sal_used = df_all[df_all.Name.isin(lock)].Salary.sum()
sal_left = 50000-sal_used
players_left = 6-len(lock)


to_remove = ['Julian Suri','Adrian Otaegui','Matt Kuchar','Fabrizio Zanotti','Jordan L. Smith',
            'Richard Sterne','Tapio Pulkkanen','Matt Jones']
to_remove = ['Martin Kaymer','Shane Lowry','Francesco Molinari','Justin Harding','Alex Noren',
            'Tom Lewis','C.T. Pan','Lee Westwood','Edoardo Molinari','Hao-Tong Li']
to_remove = ['Ian Poulter', 'Jordan L. Smith','Richard Sterne','David Lipsky']
to_remove = ['Bryson DeChambeau','Ian Poulter','Tom Lewis','Joost Luiten','Matt Wallace',
            'Graeme McDowell','Justin Harding','Martin Kaymer','Andrew Johnston']
to_remove = []

min_sal = 6000
max_sal = 12000

In [16]:
#Opt by made cut (or T10 if no cut)
opt_teams = []
df_now = df_all.copy()


#Filter down remaining players by sal
df_now = df_now[(df_now.Salary>=min_sal) &
               (df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Name.isin(lock)]

df_now = df_now[~ df_now.Name.isin(to_remove)]

num_teams = 6
for j in np.arange(num_teams):
    s = df_now.Salary.values
    c = df_now.make_cut*100
    T20 = df_now['top_20']*100
    T10 = df_now['top_10']*100
    w = df_now['win']*100
    n = len(df_now)
    #print(n,len(c),len(s))
    x = cp.Variable(n,boolean=True)
    cap = 50000
    constraints = [cp.sum(x) == players_left,
                 x*s <= sal_left]

    # Form objective.
    obj = cp.Maximize(x*c)
    #obj2 = cp.Maximize(x*T20)
    #obj3 = cp.Maximize(x*c)

    # Form and solve problem.
    #T10
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value.
    opt_team = df_now['Name'][x.value>.5].tolist()
    opt_teams.append(opt_team)
    #print('Team {}:'.format(j+1),opt_team)
    print('Team {} (obj = {:.1f}):'.format(j+1,prob.value),opt_team)

    
    #delet opt team
    #print(len(df_now))
    df_now = df_now[~df_now['Name'].isin(opt_team)]
    #print(len(df_now))

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


SolverError: 

                    CVXPY needs additional software (a `mixed-integer solver`) to handle this model.
                    The web documentation
                        https://www.cvxpy.org/tutorial/advanced/index.html#mixed-integer-programs
                    reviews open-source and commercial options for mixed-integer solvers.

                    Quick fix: if you install the python package CVXOPT (pip install cvxopt),
                    then CVXPY can use the open-source mixed-integer solver `GLPK`.
                

In [22]:
#Opt by top 20
opt_teams_T20 = []
df_now = df_all.copy()

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary>=min_sal) &
               (df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Name.isin(lock)]

df_now = df_now[~ df_now.Name.isin(to_remove)]


for j in np.arange(num_teams):
    s = df_now.Salary.values
    c = df_now.make_cut*100
    T20 = df_now['top_20']*100
    T10 = df_now['top_10']*100
    w = df_now['win']*100
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    cap = 50000
    constraints = [cp.sum(x) == players_left,
                 x*s <= sal_left]

    # Form objective.
    obj = cp.Maximize(x*T20)
    #obj2 = cp.Maximize(x*T20)
    #obj3 = cp.Maximize(x*c)

    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value.
    opt_team = df_now['Name'][x.value>.5].tolist()
    opt_teams_T20.append(opt_team)
    print('Team {} (obj = {:.1f}):'.format(j+1,prob.value),opt_team)
    
    #delet opt team
    #print(len(df_now))
    df_now = df_now[~df_now['Name'].isin(opt_team)]
    #print(len(df_now))

Team 1 (obj = 190.2): ['Thomas Pieters', 'Paul Waring', 'Benjamin Hebert', 'Romain Langasque', 'Richie Ramsay', 'Kalle Samooja']
Team 2 (obj = 170.5): ['Martin Kaymer', 'Thomas Detry', 'Renato Paratore', 'Mikko Korhonen', 'Scott Jamieson', 'Francesco Laporta']
Team 3 (obj = 145.8): ['Gavin Green', 'Adri Arnaus', 'Jason Scrivener', 'Jordan L. Smith', 'Joakim Lagergren', 'Jack Senior']
Team 4 (obj = 138.3): ['Andy Sullivan', 'Hao-Tong Li', 'George Coetzee', 'Dean Burmester', 'Matthieu Pavon', 'Jack Singh Brar']
Team 5 (obj = 126.6): ['Ross Fisher', 'Min Woo Lee', 'Sam Horsfield', 'Zander Lombard', 'Calum Hill', 'Joachim B. Hansen']
Team 6 (obj = 111.9): ['Guido Migliozzi', 'Nicolas Colsaerts', 'Aaron Rai', 'Victor Dubuisson', 'Pablo Larrazabal', 'Jeff Winther']


In [23]:
for num_teams in np.arange(1,num_teams+1):
    print('In top {} teams:'.format(num_teams))
    cut_not_t20 = set(np.hstack(opt_teams[:num_teams])) - set(np.hstack(opt_teams_T20[:num_teams]))
    t20_not_cut = set(np.hstack(opt_teams_T20[:num_teams])) - set(np.hstack(opt_teams[:num_teams]))
    print('\tIn made-cut, not Top 20:',cut_not_t20)
    print('\tIn Top 20, not made-cut:',t20_not_cut)

In top 1 teams:
	In made-cut, not Top 20: {'Martin Kaymer', 'Mikko Korhonen'}
	In Top 20, not made-cut: {'Kalle Samooja', 'Thomas Pieters'}
In top 2 teams:
	In made-cut, not Top 20: {'Joakim Lagergren'}
	In Top 20, not made-cut: {'Francesco Laporta'}
In top 3 teams:
	In made-cut, not Top 20: {'Jack Singh Brar', 'Hao-Tong Li'}
	In Top 20, not made-cut: {'Jack Senior', 'Adri Arnaus'}
In top 4 teams:
	In made-cut, not Top 20: {'Edoardo Molinari', 'Sam Horsfield'}
	In Top 20, not made-cut: {'Jack Senior', 'Matthieu Pavon'}
In top 5 teams:
	In made-cut, not Top 20: {'Edoardo Molinari', 'Aaron Rai'}
	In Top 20, not made-cut: {'Calum Hill', 'Zander Lombard'}
In top 6 teams:
	In made-cut, not Top 20: {'Edoardo Molinari'}
	In Top 20, not made-cut: {'Nicolas Colsaerts'}


In [21]:
# num_teams = 3
# cut_not_t20= set(np.hstack(opt_teams[:num_teams])) - set(np.hstack(opt_teams_T20[:num_teams]))
# print('In made-cut team, not Top 20:',cut_not_t20)

In [22]:
# print('In top 20, not made-cut:')
# print(set(np.hstack(opt_teams_T20[:num_teams])) - set(np.hstack(opt_teams[:num_teams])))

## Creat lineups as csv

In [24]:
#Assign exposures for top j teams
exposures = [50,45,40]
if len(exposures)<len(opt_teams_T20):
    exposures = exposures + [0]*(len(opt_teams_T20)-len(exposures))
exposures

[50, 45, 40, 0, 0, 0]

In [25]:
#Create df to hold player pool
df_teams = df2[['Name + ID','Name','Salary']]
df_teams.sort_values(by='Salary',ascending=False,inplace=True)
df_teams['Exposure'] = 0
df_teams.head(2)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Name + ID,Name,Salary,Exposure
0,Thomas Pieters (14344121),Thomas Pieters,10600.0,0
1,Martin Kaymer (14344122),Martin Kaymer,10200.0,0


In [26]:
#Use opt_teams or opt_teams_T20
for i, team in enumerate(opt_teams_T20):
    df_teams.Exposure = np.where(df_teams.Name.isin(team),exposures[i],df_teams.Exposure)

#Add more players
more_players = {}

#add those in made cut
num_teams_used = len([e for e in exposures if e>0])
cut_not_t20 = set(np.hstack(opt_teams[:num_teams_used])) - set(np.hstack(opt_teams_T20[:num_teams_used]))
for p in cut_not_t20:
    more_players[p] = 30
    _

#Add any additional people
# more_players['Brooks Koepka']=25
# more_players['Erik Van Rooyen'] = 40
# more_players['Mikko Korhonen'] = 40
# # more_players['Thomas Detry']=100
# # more_players['Christiaan Bezuidenhout']=100
# # more_players['Thomas Detry'] = 100
# # more_players['Christiaan Bezuidenhout'] = 100
# #Fix these clear blind spots in DG model
# more_players['Bernd Wiesberger'] = 40
# more_players['Gavin Green'] = 40
# more_players['Scott Jamieson'] = 40
# more_players['Andy Sullivan'] = 40
# more_players['Jack Singh Brar'] = 40
for p in list(more_players.keys()):
    df_teams.Exposure = np.where(df_teams.Name==p,more_players[p],df_teams.Exposure)
    
df_teams = df_teams[df_teams.Exposure > 0]
df_teams['Actual Exposure'] = 0
df_teams.reset_index(drop=True,inplace=True)
df_teams

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Name + ID,Name,Salary,Exposure,Actual Exposure
0,Thomas Pieters (14344121),Thomas Pieters,10600.0,50,0
1,Martin Kaymer (14344122),Martin Kaymer,10200.0,45,0
2,Thomas Detry (14344123),Thomas Detry,10000.0,45,0
3,Adri Arnaus (14344126),Adri Arnaus,9400.0,40,0
4,Hao-Tong Li (14344127),Hao-Tong Li,9300.0,30,0
5,Gavin Green (14344128),Gavin Green,9200.0,40,0
6,Jordan L. Smith (14344130),Jordan L. Smith,9000.0,40,0
7,Paul Waring (14344134),Paul Waring,8600.0,50,0
8,Romain Langasque (14344137),Romain Langasque,8300.0,50,0
9,Jason Scrivener (14344138),Jason Scrivener,8200.0,40,0


In [27]:
#initialize df_lineups, which will be written to csv
df_lineups = pd.DataFrame(columns=['G1','G2','G3','G4','G5','G6'])
df_lineups

,G1,G2,G3,G4,G5,G6


In [28]:
#First, need a function that picks elements from list L with probabilies probs
def pick_element_per_probs(L,probs):
    #make sure probs are normalized
    probs = [x / sum(probs) for x in probs]
    #pick a random number
    r = np.random.random()
    c = 0 #cumulative sum
    for i, prob in enumerate(probs):
        if r < prob + c:
            return L[i]
        c += prob

In [29]:
#Now start churning
#Now set parameters for number of teams and salary limits
num_teams = 44
min_sal = 49300 - sal_used
max_sal = 50000 - sal_used

#Reset df_teams
df_teams['Actual Exposure'] = 0

m = 0 #Completed teams so far
while (m < num_teams):
    #pick 6 players per probabilties
    curr_team = []
#     for p in lock_full_names:
#         curr_team.append(p)
    #print(curr_team)
    df_now = df_teams.copy()
    players = df_now['Name + ID'].tolist()
    probs = df_now['Exposure'].tolist()
    for i in range(players_left):
        p = pick_element_per_probs(players,probs)
        curr_team.append(p)
        df_now = df_now[df_now['Name + ID']!=p]
        players = df_now['Name + ID'].tolist()
        probs = df_now['Exposure'].tolist()
    #Now curr_team is list of 6 names.  Check if salary matches
    df_now = df_teams[df_teams['Name + ID'].isin(curr_team)]
    total_sal = df_now.Salary.sum()
    if (total_sal < min_sal or total_sal > max_sal):
        continue
    
#     print('Salary good for team:',curr_team)
#     print(df_now)
    #Put team in order of decreasing salary
    df_now.sort_values(by='Salary',ascending=False,inplace=True)
    #curr_team = df_now['Name + ID'].tolist()
    curr_team = lock_full_names + df_now['Name + ID'].tolist()
    #print(curr_team)
    #Check if team has shown up before in df_lineups
    df3 = (df_lineups == np.array(curr_team))
    old_team = df3.transpose().all().any() #True if team is already in df_lineups
    if old_team:
        continue
        
    #OK, so now we have a new team!  Add it to df_lineups
    df_lineups.loc[m] = curr_team
    #Also update 'Actual Exposure' in df_teams
    new_players = np.where(df_teams['Name + ID'].isin(curr_team),1,0)
    df_teams['Actual Exposure'] = df_teams['Actual Exposure'] + new_players
    
    #Finally incredment m
    m += 1
    #print('Team {}, sal = {}'.format(m,total_sal))
    
df_teams[['Name + ID','Actual Exposure',
         'Exposure','Salary']]

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Name + ID,Actual Exposure,Exposure,Salary
0,Thomas Pieters (14344121),16,50,10600.0
1,Martin Kaymer (14344122),12,45,10200.0
2,Thomas Detry (14344123),11,45,10000.0
3,Adri Arnaus (14344126),14,40,9400.0
4,Hao-Tong Li (14344127),11,30,9300.0
5,Gavin Green (14344128),10,40,9200.0
6,Jordan L. Smith (14344130),11,40,9000.0
7,Paul Waring (14344134),14,50,8600.0
8,Romain Langasque (14344137),15,50,8300.0
9,Jason Scrivener (14344138),13,40,8200.0


In [30]:
#If you're good with actual exposure, then write it!
df_lineups.to_csv('Lineups/DK Entires Euro.csv')

In [30]:
df_lineups['All'] = df_lineups.G1 + df_lineups.G2 + df_lineups.G3 + df_lineups.G4 + df_lineups.G5 + df_lineups.G6 
#df_lineups[(df_lineups.All.str.contains('Casey'))]

In [31]:
df_lineups[(df_lineups.All.str.contains('Higgs'))&
          (df_lineups.All.str.contains('Scott'))&
          (df_lineups.All.str.contains('Knox'))]

,G1,G2,G3,G4,G5,G6,All
